In [ ]:
import os
import pandas as pd
from collections import defaultdict

notebook_dir = os.path.abspath(os.path.dirname("__file__"))
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

In [ ]:
# Read the Funk dataset and clean
funk = pd.read_excel(
    os.path.join(project_root, "data/benchmarks/1-s2.0-S0092867422013599-mmc4.xlsx"),
    skiprows=1,
)
# Select only the rows where cluster is in 149, 121, 21, 167, 197, 37
funk = funk[funk["Interphase cluster"].isin([149, 121, 21, 167, 197, 37])].reset_index(
    drop=True
)
# Rename "Gene symbol" to "gene_symbol" and "Interphase cluster" to "cluster"
funk.rename(
    columns={
        "Gene symbol": "gene_symbol",
        "Interphase cluster": "cluster",
    },
    inplace=True,
)
# Drop all other columns
funk = funk[["cluster", "gene_symbol"]]
# Sort by cluster
funk.sort_values(by=["cluster"], inplace=True)
# Display the Funk dataset
display(funk)

In [ ]:
# Read the Wainberg dataset and clean
wainberg = pd.read_excel(
    os.path.join(project_root, "data/benchmarks/Supplementary_Data_2.xlsx"),
    sheet_name="Co-essential Modules",
    skiprows=(0, 1),
)
# Select Module # 2067 and 2213
wainberg = wainberg[wainberg["Module #"].isin([2067, 2213])].reset_index(drop=True)
# Create a new dataframe to store the reshaped data
reshaped_data = []
# Process each row in the original dataframe
for _, row in wainberg.iterrows():
    module_num = row["Module #"]

    # Process 'Genes' column
    if pd.notna(row["Genes"]):
        reshaped_data.append({"cluster": module_num, "gene_symbol": row["Genes"]})

    # Process all the 'Unnamed: X' columns that contain gene symbols
    for col in wainberg.columns:
        if col.startswith("Unnamed:") and pd.notna(row[col]):
            # Skip empty cells or non-gene values
            if isinstance(row[col], str) and row[col].strip() != "":
                reshaped_data.append({"cluster": module_num, "gene_symbol": row[col]})
            elif isinstance(row[col], (int, float)) and not pd.isna(row[col]):
                # Convert numeric values to string if they appear to be gene IDs
                reshaped_data.append(
                    {"cluster": module_num, "gene_symbol": str(row[col])}
                )
# Create the reshaped dataframe
wainberg = pd.DataFrame(reshaped_data)
# Sort by cluster
wainberg.sort_values(by=["cluster"], inplace=True)
# Display the Wainberg dataset
display(wainberg)

In [ ]:
schaffer = pd.read_excel(
    os.path.join(project_root, "data/benchmarks/U2OS Cell Map Assemblies.xlsx")
)
# Select Assembly ID C5255, C5415
schaffer = schaffer[schaffer["Assembly ID"].isin(["C5255", "C5415"])].reset_index(
    drop=True
)
# Create a new dataframe to store the reshaped data
reshaped_data = []

# Process each row in the original dataframe
for _, row in schaffer.iterrows():
    cluster = row["Assembly ID"]

    # Handle the case where Proteins might be missing or NaN
    if pd.notna(row["Proteins"]):
        # Split the space-separated proteins
        proteins = str(row["Proteins"]).split()

        # Add each protein to the reshaped data
        for protein in proteins:
            if protein.strip():  # Skip empty strings
                reshaped_data.append(
                    {"cluster": cluster, "gene_symbol": protein.strip()}
                )
# Create the reshaped dataframe
schaffer = pd.DataFrame(reshaped_data)
# Rename "Gene Symbol" to "gene_symbol" and "Assembly ID" to "cluster"
schaffer.rename(
    columns={
        "Gene Symbol": "gene_symbol",
        "Assembly ID": "cluster",
    },
    inplace=True,
)
# Sort by cluster
schaffer.sort_values(by=["cluster"], inplace=True)
# Display the Schaffer dataset
display(schaffer)

In [ ]:
# Read in the uniprot_data.tsv
uniprot_data = pd.read_csv(
    os.path.join(project_root, "data/benchmarks/uniprot_data.tsv"), sep="\t"
)
# Create multiple rows for each gene name with position information
expanded_rows = []
for _, row in uniprot_data.iterrows():
    if pd.notna(row["gene_names"]):
        gene_names = row["gene_names"].split()
        for position, gene in enumerate(gene_names):
            new_row = row.copy()
            new_row["gene_name"] = gene
            new_row["position"] = position
            expanded_rows.append(new_row)
# Create expanded dataframe
expanded_df = pd.DataFrame(expanded_rows)
# Sort by gene_name and position, then drop duplicates keeping the one with lower position
expanded_df = expanded_df.sort_values(["gene_name", "position"])
expanded_df = expanded_df.drop_duplicates("gene_name", keep="first")
# Select and rename columns
uniprot_data = expanded_df[["gene_name", "entry", "function"]].rename(
    columns={"function": "uniprot_function"}
)

In [ ]:
# Merge each dataset with UniProt data
for dataset_name in ["funk", "wainberg", "schaffer"]:
    dataset = locals()[dataset_name]  # Access dataset from local variables

    # Merge with UniProt data
    dataset = dataset.merge(
        uniprot_data, how="left", left_on="gene_symbol", right_on="gene_name"
    ).drop(columns="gene_name", errors="ignore")

    # Print the number of rows in the merged dataset and the number of rows missing a uniprot_function
    print(
        f"{dataset_name} dataset: {len(dataset)} rows, {dataset['uniprot_function'].isna().sum()} missing uniprot_function"
    )

    # Update the original dataset variable
    locals()[dataset_name] = dataset

In [ ]:
display(funk)
display(wainberg)
display(schaffer)

In [ ]:
from mozzarellm import analyze_gene_clusters, reshape_to_clusters
from mozzarellm.prompts import ROBUST_SCREEN_CONTEXT, ROBUST_CLUSTER_PROMPT
from mozzarellm.configs import (
    DEFAULT_OPENAI_REASONING_CONFIG,
    DEFAULT_OPENAI_CONFIG,
    DEFAULT_ANTHROPIC_CONFIG,
    DEFAULT_GEMINI_CONFIG,
)

### Reshape clusters

In [ ]:
GENE_COL = "gene_symbol"
CLUSTER_COL = "cluster"
UNIPROT_COL = "uniprot_function"

# Dictionary to store the processed results
processed_data = {}

for dataset_name in ["funk", "wainberg", "schaffer"]:
    # Get the dataset from local variables
    sample_data = locals()[dataset_name]

    # Process with reshape_to_clusters without saving
    cluster_df, gene_features = reshape_to_clusters(
        input_df=sample_data,
        gene_col=GENE_COL,
        cluster_col=CLUSTER_COL,
        uniprot_col=UNIPROT_COL,
        verbose=True,
    )

    # Store results in dictionary for later use
    processed_data[dataset_name] = {
        "clusters": cluster_df,
        "gene_features": gene_features,
    }

    # Print summary stats
    print(
        f"Processed {dataset_name}: {len(cluster_df)} clusters with {len(gene_features)} unique genes"
    )

### Knowledge cutoffs:
- o4-mini: May 2024
- o3: May 2024
- o3-mini: Sep 2023
- gpt-4.1: May 2024
- gpt-4o: Sep 2023
- claude-3-7-sonnet-20250219: Nov 2024
- claude-3-5-haiku-20241022: Apr 2024
- gemini-2.5-pro-preview-03-25: Jan 2025
- gemini-2.5-flash-preview-04-17: Jan 2025

In [ ]:
# Define the models and configurations to test
models_to_test = {
    "openai_reasoning": {
        "models": ["o4-mini", "o3-mini"],
        "config": DEFAULT_OPENAI_REASONING_CONFIG,
    },
    "openai": {"models": ["gpt-4.1", "gpt-4o"], "config": DEFAULT_OPENAI_CONFIG},
    "anthropic": {
        "models": ["claude-3-7-sonnet-20250219", "claude-3-5-haiku-20241022"],
        "config": DEFAULT_ANTHROPIC_CONFIG,
    },
    "gemini": {
        "models": ["gemini-2.5-pro-preview-03-25", "gemini-2.5-flash-preview-04-17"],
        "config": DEFAULT_GEMINI_CONFIG,
    },
}

# Dictionary to track results
results_summary = {}

# Loop through datasets
for dataset_name in ["funk", "wainberg", "schaffer"]:
    print(f"\n{'='*60}")
    print(f"Processing dataset: {dataset_name}")
    print(f"{'='*60}")

    results_summary[dataset_name] = {}

    # Get the processed data from our dictionary
    cluster_df = processed_data[dataset_name]["clusters"]
    gene_features = processed_data[dataset_name]["gene_features"]

    # Loop through each configuration and model
    for provider, provider_info in models_to_test.items():
        models = provider_info["models"]
        config_dict = provider_info["config"]

        for MODEL_NAME in models:
            print(f"\n{'-'*50}")
            print(f"Running {MODEL_NAME} on {dataset_name}")
            print(f"{'-'*50}")

            # Define results directory for this model
            RESULTS_DIR = os.path.join(
                project_root,
                f"results/{dataset_name}/{provider}/{MODEL_NAME.replace('/', '_')}",
            )
            os.makedirs(RESULTS_DIR, exist_ok=True)

            try:
                # Run the analysis with in-memory DataFrames
                results = analyze_gene_clusters(
                    # Input data options
                    input_df=cluster_df,
                    # Model and configuration
                    model_name=MODEL_NAME,
                    config_dict=config_dict,
                    # Analysis context and prompts
                    screen_context=ROBUST_SCREEN_CONTEXT,
                    cluster_analysis_prompt=ROBUST_CLUSTER_PROMPT,
                    # Gene annotations
                    gene_annotations_df=gene_features,
                    # Processing options
                    batch_size=1,
                    # Output options
                    output_file=f"{RESULTS_DIR}/{MODEL_NAME.replace('/', '_')}",
                    save_outputs=True,
                    outputs_to_generate=["json", "clusters", "flagged_genes"],
                )

                results_summary[dataset_name][f"{provider}_{MODEL_NAME}"] = "Success"
                print(
                    f"✓ Successfully analyzed {dataset_name} with {provider} {MODEL_NAME}"
                )

            except Exception as e:
                results_summary[dataset_name][f"{provider}_{MODEL_NAME}"] = (
                    f"Failed: {str(e)}"
                )
                print(
                    f"✗ Error analyzing {dataset_name} with {provider} {MODEL_NAME}: {str(e)}"
                )

# Print final summary
print("\n\n" + "=" * 80)
print("BENCHMARK RESULTS SUMMARY:")
print("=" * 80)
for dataset, model_results in results_summary.items():
    print(f"\nDataset: {dataset}")
    for model, result in model_results.items():
        status = "✓" if result == "Success" else "✗"
        print(f"  {status} {model}: {result}")

In [ ]:
import os
import pandas as pd

# Datasets to analyze
datasets = ["funk", "wainberg", "schaffer"]
# Models to check
models_to_check = [
    "claude-3-7-sonnet-20250219",
    "claude-3-5-haiku-20241022",
    "gemini-2.5-flash-preview-04-17",
    "gemini-2.5-pro-preview-03-25",
    "gpt-4.1",
    "gpt-4o",
    "o3-mini",
    "o4-mini",
]


def parse_gene_list(gene_str):
    """Parse gene list string into a list of genes, considering different separators"""
    if not gene_str or pd.isna(gene_str):
        return []

    gene_str = str(gene_str).strip()

    # Try different separators
    for separator in [";", ",", " "]:
        if separator in gene_str:
            return [g.strip() for g in gene_str.split(separator) if g.strip()]

    # If no separator found but string is not empty
    return [gene_str] if gene_str else []


def load_data(project_root):
    """Load benchmark findings and prepare data structures"""
    # Load findings.csv
    findings_path = os.path.join(project_root, "data/benchmarks/findings.csv")
    try:
        findings_df = pd.read_csv(findings_path)
    except Exception as e:
        print(f"Error loading findings file: {str(e)}")
        return None

    # Prepare benchmark data
    benchmark_data = defaultdict(dict)

    for dataset, group in findings_df.groupby("dataset"):
        if dataset not in datasets:
            continue

        print(f"\nAnalyzing dataset: {dataset}")

        # Process findings for this dataset
        for _, row in group.iterrows():
            cluster = str(row["cluster"])
            gene = row["gene"]
            function = row["function"]

            if cluster not in benchmark_data[dataset]:
                benchmark_data[dataset][cluster] = {"function": function, "genes": []}

            benchmark_data[dataset][cluster]["genes"].append(gene)

    return benchmark_data


def load_clusters_data(project_root, dataset, model):
    """Load clusters data for a specific model and dataset"""
    safe_model_name = model.replace("/", "_")
    clusters_file = os.path.join(
        project_root,
        f"results/{dataset}/{safe_model_name}/{safe_model_name}_clusters.csv",
    )

    if not os.path.exists(clusters_file):
        print(f"No clusters file found for {model} on {dataset}")
        return None

    try:
        return pd.read_csv(clusters_file)
    except Exception as e:
        print(f"Error loading clusters for {model} on {dataset}: {str(e)}")
        return None


def analyze_model_output(project_root, benchmark_data):
    """Analyze and compare model outputs with benchmark data"""
    results = defaultdict(lambda: defaultdict(dict))

    for dataset, clusters in benchmark_data.items():
        # Count benchmark clusters and genes for this dataset
        benchmark_clusters = list(clusters.keys())
        benchmark_genes = []
        for cluster_data in clusters.values():
            benchmark_genes.extend(cluster_data["genes"])
        benchmark_genes = list(set(benchmark_genes))  # Remove duplicates

        for model in models_to_check:
            print(f"  Checking model: {model}")

            # Load clusters.csv for this model and dataset
            safe_model_name = model.replace("/", "_")
            clusters_file = os.path.join(
                project_root,
                f"results/{dataset}/{safe_model_name}/{safe_model_name}_clusters.csv",
            )

            if not os.path.exists(clusters_file):
                # Record failure and continue to next model
                results[dataset][model] = create_empty_result(
                    benchmark_clusters,
                    benchmark_genes,
                    "Failed - No clusters.csv found",
                )
                continue

            try:
                clusters_df = pd.read_csv(clusters_file)
            except Exception as e:
                # Record error and continue to next model
                results[dataset][model] = create_empty_result(
                    benchmark_clusters,
                    benchmark_genes,
                    f"Failed - Error loading file: {str(e)}",
                )
                continue

            # Check for required columns
            if "cluster_id" not in clusters_df.columns:
                results[dataset][model] = create_empty_result(
                    benchmark_clusters,
                    benchmark_genes,
                    "Failed - Missing cluster_id column",
                )
                continue

            # Process model output
            results[dataset][model] = compare_model_to_benchmark(
                clusters_df, benchmark_clusters, benchmark_genes, clusters
            )

    return results


def create_empty_result(benchmark_clusters, benchmark_genes, status="Failed"):
    """Create a default result structure for failed analyses"""
    return {
        "status": status,
        "cluster_matches": {},
        "gene_categories": {},
        "total_clusters": len(benchmark_clusters),
        "found_clusters": 0,
        "found_cluster_percent": 0.0,
        "total_genes": len(benchmark_genes),
        "found_genes": 0,
        "found_gene_percent": 0.0,
        "novel_count": 0,
        "uncharacterized_count": 0,
    }


def compare_model_to_benchmark(
    clusters_df, benchmark_clusters, benchmark_genes, benchmark_data
):
    """Compare model output to benchmark data and compute metrics"""
    # Find all columns that might contain genes
    gene_columns = [
        col
        for col in clusters_df.columns
        if any(term in col.lower() for term in ["gene", "genes"])
    ]

    # Convert cluster IDs to strings for comparison
    clusters_df["cluster_id_str"] = clusters_df["cluster_id"].astype(str)

    # Initialize tracking
    cluster_matches = {}
    found_clusters = []
    gene_categories = {}
    found_genes = []  # This will now only include properly categorized genes
    novel_count = 0
    uncharacterized_count = 0
    established_count = 0

    # Check each benchmark cluster
    for benchmark_cluster, cluster_info in benchmark_data.items():
        benchmark_function = cluster_info["function"].lower()
        benchmark_genes_for_cluster = cluster_info["genes"]

        # Find matching clusters in model output
        cluster_rows = clusters_df[clusters_df["cluster_id_str"] == benchmark_cluster]

        if not cluster_rows.empty:
            # Found exact cluster match
            cluster_matches[benchmark_cluster] = "found"
            found_clusters.append(benchmark_cluster)

            # Check for function match
            function_match = check_function_match(cluster_rows, benchmark_function)
            if function_match:
                cluster_matches[benchmark_cluster] += "_with_matching_function"

            # Check each gene in this cluster
            for gene in benchmark_genes_for_cluster:
                # Get the category using the find_gene_category function
                gene_category = find_gene_category(cluster_rows, gene, gene_columns)

                # Only count genes that are properly categorized into one of the three categories
                if gene_category in ["established", "novel_role", "uncharacterized"]:
                    found_genes.append(gene)

                    # Count each category
                    if gene_category == "novel_role":
                        novel_count += 1
                    elif gene_category == "uncharacterized":
                        uncharacterized_count += 1
                    elif gene_category == "established":
                        established_count += 1

                # Always record the category for the gene categorization table
                gene_categories[gene] = gene_category
        else:
            # No exact match, check for similar function in other clusters
            function_cluster = find_function_in_other_clusters(
                clusters_df, benchmark_function
            )
            if function_cluster:
                cluster_matches[benchmark_cluster] = (
                    f"similar_function_in_{function_cluster}"
                )
            else:
                cluster_matches[benchmark_cluster] = "not_found"

    # Calculate metrics
    total_clusters = len(benchmark_clusters)
    found_cluster_count = len(found_clusters)
    found_cluster_percent = (
        (found_cluster_count / total_clusters) * 100 if total_clusters > 0 else 0
    )

    total_genes = len(benchmark_genes)
    found_gene_count = len(set(found_genes))  # Ensure uniqueness
    found_gene_percent = (
        (found_gene_count / total_genes) * 100 if total_genes > 0 else 0
    )

    # Print summary with more detailed breakdown
    print(
        f"    Found {found_cluster_count}/{total_clusters} clusters ({found_cluster_percent:.1f}%)"
    )
    print(
        f"    Properly categorized genes: {found_gene_count}/{total_genes} ({found_gene_percent:.1f}%)"
    )
    print(
        f"      Established: {established_count}, Novel role: {novel_count}, Uncharacterized: {uncharacterized_count}"
    )

    return {
        "status": "Success",
        "total_clusters": total_clusters,
        "found_clusters": found_cluster_count,
        "found_cluster_percent": found_cluster_percent,
        "cluster_matches": cluster_matches,
        "total_genes": total_genes,
        "found_genes": found_gene_count,
        "found_gene_percent": found_gene_percent,
        "established_count": established_count,
        "novel_count": novel_count,
        "uncharacterized_count": uncharacterized_count,
        "gene_categories": gene_categories,
    }


def find_gene_category(cluster_rows, gene, gene_columns):
    """Find gene in cluster and determine its category"""
    # Check each row in the cluster
    for _, row in cluster_rows.iterrows():
        # Check each specific category column
        if "novel_role_genes" in row and pd.notna(row["novel_role_genes"]):
            gene_list = parse_gene_list(row["novel_role_genes"])
            if gene in gene_list:
                return "novel_role"  # Found in novel_role category

        if "uncharacterized_genes" in row and pd.notna(row["uncharacterized_genes"]):
            gene_list = parse_gene_list(row["uncharacterized_genes"])
            if gene in gene_list:
                return "uncharacterized"  # Found in uncharacterized category

        if "established_genes" in row and pd.notna(row["established_genes"]):
            gene_list = parse_gene_list(row["established_genes"])
            if gene in gene_list:
                return "established"  # Found in established category

    # If we reach here, the gene wasn't found in any of the three category columns

    # Check if it's in any general gene column (for the gene categorization table)
    for _, row in cluster_rows.iterrows():
        for gene_col in gene_columns:
            if gene_col in row and pd.notna(row[gene_col]):
                gene_list = parse_gene_list(row[gene_col])
                if gene in gene_list:
                    return "found_but_uncategorized"  # Found in a general gene list but not categorized

    # If not found anywhere
    return "missing_in_cluster"


def generate_summary_table(results):
    """Generate enhanced summary table with strict gene categorization metrics"""
    rows = []

    for dataset, models in results.items():
        for model, data in models.items():
            # Skip failed analyses
            if data["status"] != "Success":
                row = {
                    "Dataset": dataset,
                    "Model": model,
                    "Status": data["status"],
                    "Clusters Found": f"0/{data['total_clusters']} (0.0%)",
                    "Genes Found": f"0/{data['total_genes']} (0.0%)",
                    "Established Genes": 0,
                    "Novel Role Genes": 0,
                    "Uncharacterized Genes": 0,
                    "Novel/Unchar Count": 0,
                    "Novel/Unchar %": "0.0%",
                }
                rows.append(row)
                continue

            # Calculate the sum of novel and uncharacterized genes
            novel_unchar_count = data["novel_count"] + data["uncharacterized_count"]

            # Calculate the percentage of novel and uncharacterized genes out of total genes
            novel_unchar_percent = (
                (novel_unchar_count / data["total_genes"]) * 100
                if data["total_genes"] > 0
                else 0
            )

            # Create a row with all metrics
            row = {
                "Dataset": dataset,
                "Model": model,
                "Status": data["status"],
                "Clusters Found": f"{data['found_clusters']}/{data['total_clusters']} ({data['found_cluster_percent']:.1f}%)",
                "Genes Found": f"{data['found_genes']}/{data['total_genes']} ({data['found_gene_percent']:.1f}%)",
                "Established Genes": data.get(
                    "established_count", 0
                ),  # Add established count
                "Novel Role Genes": data["novel_count"],
                "Uncharacterized Genes": data["uncharacterized_count"],
                "Novel/Unchar Count": novel_unchar_count,
                "Novel/Unchar %": f"{novel_unchar_percent:.1f}%",
            }
            rows.append(row)

    # Convert to DataFrame
    if rows:
        return pd.DataFrame(rows)
    return None


def generate_cluster_table(results, project_root):
    """Generate a cluster information table with biological process and follow-up suggestion"""
    # Collect all unique datasets and clusters
    dataset_clusters = {}
    all_models = set()

    for dataset, models in results.items():
        dataset_clusters[dataset] = set()
        for model, data in models.items():
            all_models.add(model)
            if data["status"] == "Success":
                dataset_clusters[dataset].update(data["cluster_matches"].keys())

    # Sort models for consistent column order
    sorted_models = sorted(all_models)

    # Create rows for the table
    rows = []
    for dataset, clusters in dataset_clusters.items():
        for cluster in sorted(clusters):
            # Create a row for each dataset-cluster combination
            row = {"Dataset": dataset, "Cluster": cluster}

            # Add columns for each model
            for model in sorted_models:
                if (
                    model in results[dataset]
                    and results[dataset][model]["status"] == "Success"
                ):
                    # Get the biological process and follow-up suggestion
                    process, followup = get_cluster_info(
                        project_root, dataset, model, cluster
                    )
                    row[f"{model}_biological_process"] = process
                    row[f"{model}_follow_up"] = followup
                else:
                    row[f"{model}_biological_process"] = "N/A"
                    row[f"{model}_follow_up"] = "N/A"

            rows.append(row)

    # Convert to DataFrame
    if rows:
        return pd.DataFrame(rows)
    return None


def get_cluster_info(project_root, dataset, model, cluster):
    """Get both biological process and follow-up suggestion for a specific cluster"""
    # Load the clusters file for this model using the provided function
    clusters_df = load_clusters_data(project_root, dataset, model)

    if clusters_df is None:
        return "File not found", "File not found"

    # Convert cluster IDs to strings for comparison
    clusters_df["cluster_id_str"] = clusters_df["cluster_id"].astype(str)

    # Find the matching cluster
    cluster_rows = clusters_df[clusters_df["cluster_id_str"] == cluster]

    if cluster_rows.empty:
        return "Cluster not found", "Cluster not found"

    # Get biological process
    process = "No process found"
    for column in [
        "cluster_biological_process",
        "biological_process",
        "function",
        "description",
    ]:
        if column in cluster_rows.columns and pd.notna(cluster_rows.iloc[0][column]):
            process = cluster_rows.iloc[0][column]
            break

    # Get follow-up suggestion
    followup = "No follow-up found"
    if "follow_up_suggestion" in cluster_rows.columns and pd.notna(
        cluster_rows.iloc[0]["follow_up_suggestion"]
    ):
        followup = cluster_rows.iloc[0]["follow_up_suggestion"]

    return process, followup


def generate_streamlined_gene_table(results):
    """Generate a streamlined gene categorization table as requested"""
    # Collect all unique datasets and genes
    dataset_genes = {}
    all_models = set()

    for dataset, models in results.items():
        dataset_genes[dataset] = set()
        for model, data in models.items():
            all_models.add(model)
            if data["status"] == "Success":
                dataset_genes[dataset].update(data["gene_categories"].keys())

    # Sort models for consistent column order
    sorted_models = sorted(all_models)

    # Create rows for the table
    rows = []
    for dataset, genes in dataset_genes.items():
        for gene in sorted(genes):
            # Create a row for each dataset-gene combination
            row = {"Dataset": dataset, "Gene": gene}

            # Add a column for each model
            for model in sorted_models:
                if (
                    model in results[dataset]
                    and results[dataset][model]["status"] == "Success"
                ):
                    gene_categories = results[dataset][model]["gene_categories"]
                    # Get this gene's category for this model (or 'missing_cluster' if not found)
                    category = gene_categories.get(gene, "missing_from_all_clusters")
                    row[model] = category
                else:
                    row[model] = "N/A"  # Model analysis failed

            rows.append(row)

    # Convert to DataFrame
    if rows:
        return pd.DataFrame(rows)
    return None


def generate_reports(project_root, results):
    """Generate all summary reports"""
    # Basic summary table
    summary_df = generate_summary_table(results)
    if summary_df is not None:
        print("\n=== SUMMARY TABLE ===")
        print(summary_df.to_string())

    # Generate streamlined gene categorization table
    gene_table = generate_streamlined_gene_table(results)
    if gene_table is not None:
        print("\n=== GENE CATEGORIZATION TABLE ===")
        print(gene_table.to_string())

    # Generate cluster information table
    cluster_table = generate_cluster_table(results, project_root)
    if cluster_table is not None:
        print("\n=== CLUSTER INFORMATION TABLE ===")
        print(cluster_table.to_string())

    return summary_df, gene_table, cluster_table

In [ ]:
# Load benchmark data
benchmark_data = load_data(project_root)

# Analyze model outputs
results = analyze_model_output(project_root, benchmark_data)

# Generate and save reports
summary_df, gene_table, cluster_table = generate_reports(project_root, results)

print("\nAnalysis complete!")

In [ ]:
# Remove any quotes in the tables
def clean_table(df):
    """Remove quotes from all string columns in the DataFrame"""
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.replace('"', "", regex=False)
        df[col] = df[col].str.replace("'", "", regex=False)
    return df


# Clean the tables
summary_df = clean_table(summary_df)
gene_table = clean_table(gene_table)
cluster_table = clean_table(cluster_table)

In [ ]:
# save all the tables to excel sheets
output_file = os.path.join(project_root, "data/benchmarks/analysis_results.xlsx")
with pd.ExcelWriter(output_file) as writer:
    summary_df.to_excel(writer, sheet_name="Summary", index=False)
    gene_table.to_excel(writer, sheet_name="Gene Categorization", index=False)
    cluster_table.to_excel(writer, sheet_name="Cluster Information", index=False)
print(f"Results saved to {output_file}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.ticker import MaxNLocator
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors

# Extract percentages from strings
summary_df["Novel/Unchar %"] = (
    summary_df["Novel/Unchar %"].str.rstrip("%").astype(float)
)

# Extract Genes Found percentage from the string format "X/Y (Z%)"
summary_df["Genes Found %"] = summary_df["Genes Found"].apply(
    lambda x: float(x.split("(")[1].split("%")[0])
)

# Create a model type mapping
model_types = {
    "claude-3-7-sonnet-20250219": "Claude",
    "claude-3-5-haiku-20241022": "Claude",
    "gemini-2.5-flash-preview-04-17": "Gemini",
    "gemini-2.5-pro-preview-03-25": "Gemini",
    "gpt-4.1": "GPT",
    "gpt-4o": "GPT",
    "o3-mini": "GPT",
    "o4-mini": "GPT",
}

# Add model type column
summary_df["Model Type"] = summary_df["Model"].map(model_types)

# Define base colors for each model type
type_base_colors = {
    "Claude": "#9467bd",  # Purple
    "Gemini": "#2ca02c",  # Green
    "GPT": "#d62728",  # Red
}

# Group models by type
models_by_type = {}
for model, model_type in model_types.items():
    if model_type not in models_by_type:
        models_by_type[model_type] = []
    models_by_type[model_type].append(model)

# Create different shades for each model within the same family
model_colors = {}
for model_type, models in models_by_type.items():
    base_color = mcolors.to_rgb(type_base_colors[model_type])

    # Create shades based on number of models
    if len(models) == 2:
        # For 2 models: one darker, one lighter
        model_colors[models[0]] = mcolors.to_hex(
            [c * 0.7 for c in base_color]
        )  # Darker
        model_colors[models[1]] = mcolors.to_hex(
            [min(1.0, c * 1.3) for c in base_color]
        )  # Lighter
    elif len(models) == 4:  # For GPT with 4 models
        # Create 4 shades from dark to light
        shadings = [0.6, 0.8, 1.0, 1.2]
        for i, model in enumerate(models):
            shade_factor = shadings[i]
            model_colors[model] = mcolors.to_hex(
                [min(1.0, c * shade_factor) for c in base_color]
            )
    else:
        # If just one model, use the base color
        for model in models:
            model_colors[model] = type_base_colors[model_type]

# Set the style
sns.set(style="whitegrid")

# Create a figure with two panels
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), sharex=True)

# Panel 1: Novel/Uncharacterized Genes
sns.barplot(
    data=summary_df,
    x="Dataset",
    y="Novel/Unchar %",
    hue="Model",
    palette=model_colors,
    ax=ax1,
)

# Set the title and labels for panel 1
ax1.set_title("Novel/Uncharacterized Gene Discovery by Model and Dataset", fontsize=16)
ax1.set_ylabel("Novel/Uncharacterized Genes (%)", fontsize=14)
ax1.set_ylim(0, 100)  # Slightly higher to give space at the top
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x)}%"))
ax1.yaxis.set_major_locator(MaxNLocator(nbins=10, integer=True))
ax1.get_legend().remove()

# Panel 2: Genes Found
sns.barplot(
    data=summary_df,
    x="Dataset",
    y="Genes Found %",
    hue="Model",
    palette=model_colors,
    ax=ax2,
)

# Set the title and labels for panel 2
ax2.set_title("Genes Found by Model and Dataset", fontsize=16)
ax2.set_xlabel("Dataset", fontsize=14)
ax2.set_ylabel("Genes Found (%)", fontsize=14)
ax2.set_ylim(0, 100)  # Slightly higher to give space at the top
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x)}%"))
ax2.yaxis.set_major_locator(MaxNLocator(nbins=10, integer=True))
ax2.get_legend().remove()

# Create custom legend
legend_elements = []

# Add model elements - group by model type for organization
for model_type in type_base_colors.keys():
    # Add model type header
    legend_elements.append(
        Line2D([0], [0], color="white", lw=0, label=f"{model_type} Models:")
    )

    # Add models of this type
    for model in sorted(models_by_type[model_type]):
        color = model_colors[model]
        # Create a simplified model name for display
        if "claude" in model:
            display_name = (
                model.replace("claude-", "Claude ")
                .replace("-20250219", "")
                .replace("-20241022", "")
            )
        elif "gemini" in model:
            display_name = (
                model.replace("gemini-", "Gemini ")
                .replace("-preview-04-17", "")
                .replace("-preview-03-25", "")
            )
        elif "gpt" in model:
            display_name = model.replace("gpt-", "GPT ")
        else:
            display_name = model

        legend_elements.append(Line2D([0], [0], color=color, lw=4, label=display_name))

    # Add a separator after each group
    if model_type != list(type_base_colors.keys())[-1]:
        legend_elements.append(Line2D([0], [0], color="white", lw=0, label=""))

# Add the legend outside the plot area, positioned between the two panels
fig.legend(
    handles=legend_elements, loc="center right", bbox_to_anchor=(0.98, 0.5), fontsize=10
)

# Adjust layout for the legend and spacing between panels
plt.tight_layout()
plt.subplots_adjust(right=0.8, hspace=0.3)

plt.show()